Part 1 - Q2 - Classic models Features Extraction
===

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import neurokit2 as nk
import tsfresh as tsf
from tsfresh import extract_features
import warnings

In [3]:
%matplotlib inline
import torch
import torch.nn.functional as F

In [4]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc

In [5]:
import param
import panel as pn
pn.extension()
import datetime as dt
import importlib
from importlib import reload
from tqdm import tqdm

In [6]:
import graph as gr
import params as par
import py_lib as lib
reload(gr)
reload(par)
reload(lib)
import graph as gr
import params as par
import py_lib as lib

In [7]:
import custom_ecg_delineate as custom
reload(custom)
import custom_ecg_delineate as custom

In [8]:
format_dict = par.create_format_dict()
format_dict

{'Age': '{:.0f}',
 'RestingBP': '{:.0f}',
 'Cholesterol': '{:.0f}',
 'MaxHR': '{:.0f}',
 'cardinality': '{:.0f}',
 'FastingBS': '{:.1f}',
 'HeartDisease': '{:.1f}',
 'Oldpeak': '{:.2f}',
 '0': '{:.0%}',
 '1': '{:.0%}',
 '1.0': '{:.0%}',
 '0.0': '{:.0%}',
 'Accuracy': '{:.1%}',
 'Balanced Accuracy': '{:.1%}',
 'F1 Score': '{:.1%}'}

In [9]:
class NotebookParams(param.Parameterized):
    test_size                			= param.Number(0.2, bounds=(0.1,0.9), doc="Test size for train_test_split")
    random_state_test_train_split = param.Integer(42)
base = NotebookParams()

# PTB dataset

## Import Data

In [48]:
# df_heart = pd.read_csv('../data_proj2/ptbdb_train.csv')
df_heart_test = pd.read_csv('../data_proj2/ptbdb_test.csv', header=None)
df_heart_train = pd.read_csv('../data_proj2/ptbdb_train.csv', header=None)

In [49]:
n_train_examples = df_heart_train.shape[0]
n_test_examples = df_heart_test.shape[0]
n_total_examples = n_train_examples + n_test_examples
print(f'The test split has {n_test_examples} rows, or {n_test_examples/n_total_examples:.2%} of all examples (which represent {n_total_examples} rows).')

The test split has 2911 rows, or 20.00% of all examples (which represent 14552 rows).


## Labels

In [50]:
label_col = list(df_heart_train.columns)[-1]
labels_count = df_heart_train[label_col].value_counts(normalize=True).to_frame().T
labels_count.columns = [str(x) for x in labels_count.columns]
labels_count.style.format(format_dict)

,1.0,0.0
proportion,72%,28%


In [51]:
pn.Column(pn.Row(base.param.add_one_hot_encoded, base.param.stand_features, base.param.test_size))

BokehModel(combine_events=True, render_bundle={'docs_json': {'c8d74110-8f61-4dc7-815d-ecec9617aeef': {'version…

## Processing ts fresh features

### For training

In [52]:
dir_path = '../output/cs_files_tsfresh/ptb/train/all_features/'
df_of_interest = df_heart_train

In [53]:
df_of_interest.head(3)

,0,1,2,3,4,5,6,7,8,9,...,178,179,180,181,182,183,184,185,186,187
0,1.000000,0.730599,0.472859,0.139526,0.000000,0.020507,0.112183,0.162847,0.180941,0.192199,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1.000000,0.152446,0.011377,0.308305,0.153584,0.096701,0.083049,0.086462,0.105802,0.071672,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.975917,0.693234,0.241399,0.107225,0.098050,0.072248,0.062500,0.054472,0.054472,0.045872,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
import py_lib as lib
reload(lib)
import py_lib as lib

In [55]:
x_train_0, x_test_0, y_train_0, y_test_0 = lib.pre_process_ecg(
        df_of_interest,
        label_col_pos=-1,
        test_size=base.test_size,
        random_state=base.random_state_test_train_split,
        split_data=False,
)

In [62]:
x_train_0_transfo = x_train_0.T.copy()
lib.tsfresh_features_extraction_loop(
        x_train_0_transfo,
        dir_path=dir_path + 'extracted_features',
        starting_point=10200,
        window_size=200,
				n_loops=8,
        only_relevant=False,
        y_vector=y_train_0[187],
)

Feature Extraction: 100%|██████████| 21/21 [00:04<00:00,  4.74it/s]


Checking that we have indeed the correct number of rows.

In [63]:
tsf_fresh_features = lib.join_all_features_csv_batches(
    dir_path=dir_path,
    pull_all=True,
)

../output/cs_files_tsfresh/ptb/train/all_features/*.csv
../output/cs_files_tsfresh/ptb/train/all_features/extracted_features_3600_3799.csv
../output/cs_files_tsfresh/ptb/train/all_features/extracted_features_3000_3199.csv
../output/cs_files_tsfresh/ptb/train/all_features/extracted_features_1800_1999.csv
../output/cs_files_tsfresh/ptb/train/all_features/extracted_features_4200_4399.csv
../output/cs_files_tsfresh/ptb/train/all_features/extracted_features_8600_8799.csv
../output/cs_files_tsfresh/ptb/train/all_features/extracted_features_4400_4599.csv
../output/cs_files_tsfresh/ptb/train/all_features/extracted_features_200_399.csv
../output/cs_files_tsfresh/ptb/train/all_features/extracted_features_7200_7399.csv
../output/cs_files_tsfresh/ptb/train/all_features/extracted_features_5000_5199.csv
../output/cs_files_tsfresh/ptb/train/all_features/extracted_features_10400_10599.csv
../output/cs_files_tsfresh/ptb/train/all_features/extracted_features_7600_7799.csv
../output/cs_files_tsfresh/ptb/

In [64]:
tsf_fresh_features.head()

,variance_larger_than_standard_deviation,has_duplicate_max,has_duplicate_min,has_duplicate,sum_values,abs_energy,mean_abs_change,mean_change,mean_second_derivative_central,median,...,fourier_entropy__bins_5,fourier_entropy__bins_10,fourier_entropy__bins_100,permutation_entropy__dimension_3__tau_1,permutation_entropy__dimension_4__tau_1,permutation_entropy__dimension_5__tau_1,permutation_entropy__dimension_6__tau_1,permutation_entropy__dimension_7__tau_1,query_similarity_count__query_None__threshold_0.0,mean_n_absolute_max__number_of_maxima_7
heartbeat_idx,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,1.0,1.0,31.761560,8.927965,0.014062,-0.005376,0.000728,0.178528,...,0.161651,0.316683,0.974941,1.566183,2.558113,3.418415,3.910249,4.101044,NaN,0.602734
1,0.0,0.0,1.0,1.0,21.162685,7.935466,0.032056,-0.005376,0.002291,0.000000,...,0.117714,0.176399,1.274168,1.105742,1.719584,2.258974,2.581583,2.731673,NaN,0.669755
2,0.0,0.0,1.0,1.0,18.361812,5.907348,0.022110,-0.005247,0.000764,0.071674,...,0.853715,1.276388,2.616993,1.230186,1.766055,2.281631,2.684533,2.957541,NaN,0.692661
3,0.0,0.0,1.0,1.0,17.381913,5.044730,0.023695,-0.005121,0.000986,0.069616,...,0.381096,0.862994,2.195313,1.524769,2.514057,3.325593,3.828358,4.140207,NaN,0.642160
4,0.0,0.0,1.0,1.0,54.557505,22.240158,0.026340,-0.005345,0.001266,0.352827,...,0.258354,0.483485,1.715575,1.569708,2.535120,3.408794,3.905917,4.161155,NaN,0.709088


In [65]:
print(f'Comparing the # heartbeats in the original dataset and in the processed one')
print(f'Originally: {df_of_interest.shape[0]} heartbeats')
print(f'After ts fresh processing: {tsf_fresh_features.shape[0]} heartbeats')
print(f'Is it the same? {tsf_fresh_features.shape[0] == df_of_interest.shape[0]}')

Comparing the # heartbeats in the original dataset and in the processed one
Originally: 11641 heartbeats
After ts fresh processing: 11641 heartbeats
Is it the same? True


### For Eval

In [66]:
dir_path = '../output/cs_files_tsfresh/ptb/eval/all_features/'
df_of_interest = df_heart_test

In [67]:
df_of_interest.head(3)

,0,1,2,3,4,5,6,7,8,9,...,178,179,180,181,182,183,184,185,186,187
0,0.951937,1.000000,0.817437,0.414680,0.163562,0.033159,0.093517,0.152012,0.150149,0.146051,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.000000,0.664604,0.406382,0.172582,0.175513,0.116574,0.079779,0.070987,0.055357,0.051449,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1.000000,0.764805,0.373942,0.173858,0.018613,0.061337,0.145939,0.150169,0.156514,0.168359,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [69]:
df_of_interest.shape

(2911, 188)

In [71]:
x_train_0, x_test_0, y_train_0, y_test_0 = lib.pre_process_ecg(
        df_of_interest,
        label_col_pos=-1,
        test_size=base.test_size,
        random_state=base.random_state_test_train_split,
        split_data=False,
)

In [72]:
x_train_0_transfo = x_train_0.T.copy()
lib.tsfresh_features_extraction_loop(
        x_train_0_transfo,
        dir_path=dir_path + 'extracted_features',
        starting_point=0,
        window_size=200,
				n_loops=15,
        only_relevant=False,
        y_vector=y_train_0[187],
)

Feature Extraction: 100%|██████████| 28/28 [00:11<00:00,  2.52it/s]


Checking that we have indeed the correct number of rows.

In [ ]:
tsf_fresh_features = lib.join_all_features_csv_batches(
    dir_path=dir_path,
    pull_all=True,
)

../output/cs_files_tsfresh/*.csv
<class 'list'>
../output/cs_files_tsfresh/extracted_features_9550_9599.csv
../output/cs_files_tsfresh/extracted_features_9550_9599.csv


../output/cs_files_tsfresh/extracted_features_4700_4749.csv
../output/cs_files_tsfresh/extracted_features_5900_5949.csv
../output/cs_files_tsfresh/extracted_features_6200_6249.csv
../output/cs_files_tsfresh/extracted_features_11250_11299.csv
../output/cs_files_tsfresh/extracted_features_400_449.csv
../output/cs_files_tsfresh/extracted_features_5500_5549.csv
../output/cs_files_tsfresh/extracted_features_10550_10599.csv
../output/cs_files_tsfresh/extracted_features_1550_1599.csv
../output/cs_files_tsfresh/extracted_features_6700_6749.csv
../output/cs_files_tsfresh/extracted_features_7000_7049.csv
../output/cs_files_tsfresh/extracted_features_1250_1299.csv
../output/cs_files_tsfresh/extracted_features_8600_8649.csv
../output/cs_files_tsfresh/extracted_features_900_949.csv
../output/cs_files_tsfresh/extracted_features_8250_8299.csv
../output/cs_files_tsfresh/extracted_features_8000_8049.csv
../output/cs_files_tsfresh/extracted_features_10500_10549.csv
../output/cs_files_tsfresh/extracted_f

In [ ]:
tsf_fresh_features.head(2)

,variance_larger_than_standard_deviation,has_duplicate_max,has_duplicate_min,has_duplicate,sum_values,abs_energy,mean_abs_change,mean_change,mean_second_derivative_central,median,...,fourier_entropy__bins_5,fourier_entropy__bins_10,fourier_entropy__bins_100,permutation_entropy__dimension_3__tau_1,permutation_entropy__dimension_4__tau_1,permutation_entropy__dimension_5__tau_1,permutation_entropy__dimension_6__tau_1,permutation_entropy__dimension_7__tau_1,query_similarity_count__query_None__threshold_0.0,mean_n_absolute_max__number_of_maxima_7
heartbeat_idx,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,1.0,1.0,31.761560,8.927965,0.014062,-0.005376,0.000728,0.178528,...,0.161651,0.316683,0.974941,1.566183,2.558113,3.418415,3.910249,4.101044,NaN,0.602734
1,0.0,0.0,1.0,1.0,21.162685,7.935466,0.032056,-0.005376,0.002291,0.000000,...,0.117714,0.176399,1.274168,1.105742,1.719584,2.258974,2.581583,2.731673,NaN,0.669755
2,0.0,0.0,1.0,1.0,18.361812,5.907348,0.022110,-0.005247,0.000764,0.071674,...,0.853715,1.276388,2.616993,1.230186,1.766055,2.281631,2.684533,2.957541,NaN,0.692661
3,0.0,0.0,1.0,1.0,17.381913,5.044730,0.023695,-0.005121,0.000986,0.069616,...,0.381096,0.862994,2.195313,1.524769,2.514057,3.325593,3.828358,4.140207,NaN,0.642160
4,0.0,0.0,1.0,1.0,54.557505,22.240158,0.026340,-0.005345,0.001266,0.352827,...,0.258354,0.483485,1.715575,1.569708,2.535120,3.408794,3.905917,4.161155,NaN,0.709088


In [ ]:
print(f'Comparing the # heartbeats in the original dataset and in the processed one')
print(f'Originally: {df_of_interest.shape[0]} heartbeats')
print(f'After ts fresh processing: {tsf_fresh_features.shape[0]} heartbeats')
print(f'Is it the same? {tsf_fresh_features.shape[0] == df_of_interest.shape[0]}')

(11641, 783)

# MIT BIH dataset

## Import Data

In [ ]:
# df_heart = pd.read_csv('../data_proj2/ptbdb_train.csv')
df_heart_test = pd.read_csv('../data_proj2/mitbih_test.csv', header=None)
df_heart_train = pd.read_csv('../data_proj2/mitbih_train.csv', header=None)

In [ ]:
n_train_examples = df_heart_train.shape[0]
n_test_examples = df_heart_test.shape[0]
n_total_examples = n_train_examples + n_test_examples
print(f'The test split has {n_test_examples} rows, or {n_test_examples/n_total_examples:.2%} of all examples (which represent {n_total_examples} rows).')

The test split has 2911 rows, or 20.00% of all examples (which represent 14552 rows).


## Labels

In [ ]:
label_col = list(df_heart_train.columns)[-1]
labels_count = df_heart_train[label_col].value_counts(normalize=True).to_frame().T
labels_count.columns = [str(x) for x in labels_count.columns]
labels_count.style.format(format_dict)

,1.0,0.0
proportion,72%,28%


In [ ]:
pn.Column(pn.Row(base.param.add_one_hot_encoded, base.param.stand_features, base.param.test_size))

BokehModel(combine_events=True, render_bundle={'docs_json': {'54d3932c-094f-4034-b18f-66649bb5a05f': {'version…

## Processing ts fresh features

### For training

In [ ]:
dir_path = '../output/cs_files_tsfresh/mitbih/train/all_features/'
df_of_interest = df_heart_train

In [ ]:
df_of_interest.head(3)

,0,1,2,3,4,5,6,7,8,9,...,178,179,180,181,182,183,184,185,186,187
0,1.000000,0.730599,0.472859,0.139526,0.000000,0.020507,0.112183,0.162847,0.180941,0.192199,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1.000000,0.152446,0.011377,0.308305,0.153584,0.096701,0.083049,0.086462,0.105802,0.071672,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.975917,0.693234,0.241399,0.107225,0.098050,0.072248,0.062500,0.054472,0.054472,0.045872,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
import py_lib as lib
reload(lib)
import py_lib as lib

In [ ]:
x_train_0, x_test_0, y_train_0, y_test_0 = lib.pre_process_ecg(
        df_of_interest,
        label_col_pos=-1,
        test_size=base.test_size,
        random_state=base.random_state_test_train_split,
        split_data=False,
)

In [ ]:
x_train_0_transfo = x_train_0.T.copy()
lib.tsfresh_features_extraction_loop(
        x_train_0_transfo,
        dir_path=dir_path + 'extracted_features',
        starting_point=400,
        window_size=200,
				n_loops=200,
        only_relevant=False,
        y_vector=y_train_0[187],
)

Checking that we have indeed the correct number of rows.

In [ ]:
tsf_fresh_features = lib.join_all_features_csv_batches(
    dir_path=dir_path,
    pull_all=True,
)

../output/cs_files_tsfresh/*.csv
<class 'list'>
../output/cs_files_tsfresh/extracted_features_9550_9599.csv
../output/cs_files_tsfresh/extracted_features_9550_9599.csv


../output/cs_files_tsfresh/extracted_features_4700_4749.csv
../output/cs_files_tsfresh/extracted_features_5900_5949.csv
../output/cs_files_tsfresh/extracted_features_6200_6249.csv
../output/cs_files_tsfresh/extracted_features_11250_11299.csv
../output/cs_files_tsfresh/extracted_features_400_449.csv
../output/cs_files_tsfresh/extracted_features_5500_5549.csv
../output/cs_files_tsfresh/extracted_features_10550_10599.csv
../output/cs_files_tsfresh/extracted_features_1550_1599.csv
../output/cs_files_tsfresh/extracted_features_6700_6749.csv
../output/cs_files_tsfresh/extracted_features_7000_7049.csv
../output/cs_files_tsfresh/extracted_features_1250_1299.csv
../output/cs_files_tsfresh/extracted_features_8600_8649.csv
../output/cs_files_tsfresh/extracted_features_900_949.csv
../output/cs_files_tsfresh/extracted_features_8250_8299.csv
../output/cs_files_tsfresh/extracted_features_8000_8049.csv
../output/cs_files_tsfresh/extracted_features_10500_10549.csv
../output/cs_files_tsfresh/extracted_f

In [ ]:
tsf_fresh_features.head()

,variance_larger_than_standard_deviation,has_duplicate_max,has_duplicate_min,has_duplicate,sum_values,abs_energy,mean_abs_change,mean_change,mean_second_derivative_central,median,...,fourier_entropy__bins_5,fourier_entropy__bins_10,fourier_entropy__bins_100,permutation_entropy__dimension_3__tau_1,permutation_entropy__dimension_4__tau_1,permutation_entropy__dimension_5__tau_1,permutation_entropy__dimension_6__tau_1,permutation_entropy__dimension_7__tau_1,query_similarity_count__query_None__threshold_0.0,mean_n_absolute_max__number_of_maxima_7
heartbeat_idx,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,1.0,1.0,31.761560,8.927965,0.014062,-0.005376,0.000728,0.178528,...,0.161651,0.316683,0.974941,1.566183,2.558113,3.418415,3.910249,4.101044,NaN,0.602734
1,0.0,0.0,1.0,1.0,21.162685,7.935466,0.032056,-0.005376,0.002291,0.000000,...,0.117714,0.176399,1.274168,1.105742,1.719584,2.258974,2.581583,2.731673,NaN,0.669755
2,0.0,0.0,1.0,1.0,18.361812,5.907348,0.022110,-0.005247,0.000764,0.071674,...,0.853715,1.276388,2.616993,1.230186,1.766055,2.281631,2.684533,2.957541,NaN,0.692661
3,0.0,0.0,1.0,1.0,17.381913,5.044730,0.023695,-0.005121,0.000986,0.069616,...,0.381096,0.862994,2.195313,1.524769,2.514057,3.325593,3.828358,4.140207,NaN,0.642160
4,0.0,0.0,1.0,1.0,54.557505,22.240158,0.026340,-0.005345,0.001266,0.352827,...,0.258354,0.483485,1.715575,1.569708,2.535120,3.408794,3.905917,4.161155,NaN,0.709088


In [ ]:
print(f'Comparing the # heartbeats in the original dataset and in the processed one')
print(f'Originally: {df_of_interest.shape[0]} heartbeats')
print(f'After ts fresh processing: {tsf_fresh_features.shape[0]} heartbeats')
print(f'Is it the same? {tsf_fresh_features.shape[0] == df_of_interest.shape[0]}')

(11641, 783)

### For Eval

In [ ]:
dir_path = '../output/cs_files_tsfresh/mitbih/eval/all_features/'
df_of_interest = df_heart_test

In [ ]:
df_of_interest.head(3)

,0,1,2,3,4,5,6,7,8,9,...,178,179,180,181,182,183,184,185,186,187
0,1.000000,0.730599,0.472859,0.139526,0.000000,0.020507,0.112183,0.162847,0.180941,0.192199,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1.000000,0.152446,0.011377,0.308305,0.153584,0.096701,0.083049,0.086462,0.105802,0.071672,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.975917,0.693234,0.241399,0.107225,0.098050,0.072248,0.062500,0.054472,0.054472,0.045872,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
x_train_0, x_test_0, y_train_0, y_test_0 = lib.pre_process_ecg(
        df_of_interest,
        label_col_pos=-1,
        test_size=base.test_size,
        random_state=base.random_state_test_train_split,
        split_data=False,
)

In [ ]:
x_train_0_transfo = x_train_0.T.copy()
lib.tsfresh_features_extraction_loop(
        x_train_0_transfo,
        dir_path=dir_path + 'extracted_features',
        starting_point=400,
        window_size=200,
				n_loops=200,
        only_relevant=False,
        y_vector=y_train_0[187],
)

Checking that we have indeed the correct number of rows.

In [ ]:
tsf_fresh_features = lib.join_all_features_csv_batches(
    dir_path=dir_path,
    pull_all=True,
)

../output/cs_files_tsfresh/*.csv
<class 'list'>
../output/cs_files_tsfresh/extracted_features_9550_9599.csv
../output/cs_files_tsfresh/extracted_features_9550_9599.csv


../output/cs_files_tsfresh/extracted_features_4700_4749.csv
../output/cs_files_tsfresh/extracted_features_5900_5949.csv
../output/cs_files_tsfresh/extracted_features_6200_6249.csv
../output/cs_files_tsfresh/extracted_features_11250_11299.csv
../output/cs_files_tsfresh/extracted_features_400_449.csv
../output/cs_files_tsfresh/extracted_features_5500_5549.csv
../output/cs_files_tsfresh/extracted_features_10550_10599.csv
../output/cs_files_tsfresh/extracted_features_1550_1599.csv
../output/cs_files_tsfresh/extracted_features_6700_6749.csv
../output/cs_files_tsfresh/extracted_features_7000_7049.csv
../output/cs_files_tsfresh/extracted_features_1250_1299.csv
../output/cs_files_tsfresh/extracted_features_8600_8649.csv
../output/cs_files_tsfresh/extracted_features_900_949.csv
../output/cs_files_tsfresh/extracted_features_8250_8299.csv
../output/cs_files_tsfresh/extracted_features_8000_8049.csv
../output/cs_files_tsfresh/extracted_features_10500_10549.csv
../output/cs_files_tsfresh/extracted_f

In [ ]:
tsf_fresh_features.head(2)

,variance_larger_than_standard_deviation,has_duplicate_max,has_duplicate_min,has_duplicate,sum_values,abs_energy,mean_abs_change,mean_change,mean_second_derivative_central,median,...,fourier_entropy__bins_5,fourier_entropy__bins_10,fourier_entropy__bins_100,permutation_entropy__dimension_3__tau_1,permutation_entropy__dimension_4__tau_1,permutation_entropy__dimension_5__tau_1,permutation_entropy__dimension_6__tau_1,permutation_entropy__dimension_7__tau_1,query_similarity_count__query_None__threshold_0.0,mean_n_absolute_max__number_of_maxima_7
heartbeat_idx,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,1.0,1.0,31.761560,8.927965,0.014062,-0.005376,0.000728,0.178528,...,0.161651,0.316683,0.974941,1.566183,2.558113,3.418415,3.910249,4.101044,NaN,0.602734
1,0.0,0.0,1.0,1.0,21.162685,7.935466,0.032056,-0.005376,0.002291,0.000000,...,0.117714,0.176399,1.274168,1.105742,1.719584,2.258974,2.581583,2.731673,NaN,0.669755
2,0.0,0.0,1.0,1.0,18.361812,5.907348,0.022110,-0.005247,0.000764,0.071674,...,0.853715,1.276388,2.616993,1.230186,1.766055,2.281631,2.684533,2.957541,NaN,0.692661
3,0.0,0.0,1.0,1.0,17.381913,5.044730,0.023695,-0.005121,0.000986,0.069616,...,0.381096,0.862994,2.195313,1.524769,2.514057,3.325593,3.828358,4.140207,NaN,0.642160
4,0.0,0.0,1.0,1.0,54.557505,22.240158,0.026340,-0.005345,0.001266,0.352827,...,0.258354,0.483485,1.715575,1.569708,2.535120,3.408794,3.905917,4.161155,NaN,0.709088


In [ ]:
print(f'Comparing the # heartbeats in the original dataset and in the processed one')
print(f'Originally: {df_of_interest.shape[0]} heartbeats')
print(f'After ts fresh processing: {tsf_fresh_features.shape[0]} heartbeats')
print(f'Is it the same? {tsf_fresh_features.shape[0] == df_of_interest.shape[0]}')

(11641, 783)